<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#What-is-Text-Fabric?" data-toc-modified-id="What-is-Text-Fabric?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>What is Text-Fabric?</a></span><ul class="toc-item"><li><span><a href="#Cuneiform-tablets-in-ATF" data-toc-modified-id="Cuneiform-tablets-in-ATF-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Cuneiform tablets in ATF</a></span></li><li><span><a href="#Text-Fabric" data-toc-modified-id="Text-Fabric-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Text-Fabric</a></span></li></ul></li><li><span><a href="#Overview" data-toc-modified-id="Overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#More-information" data-toc-modified-id="More-information-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>More information</a></span></li><li><span><a href="#Installing-Text-Fabric" data-toc-modified-id="Installing-Text-Fabric-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Installing Text-Fabric</a></span><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#TF-itself" data-toc-modified-id="TF-itself-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>TF itself</a></span></li><li><span><a href="#Cuneiform-data" data-toc-modified-id="Cuneiform-data-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Cuneiform data</a></span><ul class="toc-item"><li><span><a href="#Tip" data-toc-modified-id="Tip-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Tip</a></span></li></ul></li></ul></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#A-tour-around-the-TF-API" data-toc-modified-id="A-tour-around-the-TF-API-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>A tour around the TF API</a></span><ul class="toc-item"><li><span><a href="#Sections:-tablet---column---line" data-toc-modified-id="Sections:-tablet---column---line-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Sections: tablet - column - line</a></span></li><li><span><a href="#Navigation" data-toc-modified-id="Navigation-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Navigation</a></span><ul class="toc-item"><li><span><a href="#Source-lines" data-toc-modified-id="Source-lines-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Source lines</a></span></li><li><span><a href="#Source-lines:-refined" data-toc-modified-id="Source-lines:-refined-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Source lines: refined</a></span></li><li><span><a href="#Lines-and-cases" data-toc-modified-id="Lines-and-cases-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Lines and cases</a></span></li><li><span><a href="#Line-numbers" data-toc-modified-id="Line-numbers-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Line numbers</a></span></li></ul></li><li><span><a href="#Mass-operations" data-toc-modified-id="Mass-operations-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Mass operations</a></span></li><li><span><a href="#Walking-all-nodes" data-toc-modified-id="Walking-all-nodes-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Walking all nodes</a></span></li><li><span><a href="#All-signs" data-toc-modified-id="All-signs-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>All signs</a></span></li><li><span><a href="#Particular-signs" data-toc-modified-id="Particular-signs-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>Particular signs</a></span></li><li><span><a href="#From-instance-to-context" data-toc-modified-id="From-instance-to-context-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>From instance to context</a></span></li></ul></li><li><span><a href="#End" data-toc-modified-id="End-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>End</a></span></li><li><span><a href="#Count-all-nodes" data-toc-modified-id="Count-all-nodes-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Count all nodes</a></span></li><li><span><a href="#What-are-all-those-nodes?" data-toc-modified-id="What-are-all-those-nodes?-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>What are all those nodes?</a></span></li><li><span><a href="#Count-individual-object-types" data-toc-modified-id="Count-individual-object-types-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Count individual object types</a></span></li><li><span><a href="#Going-up" data-toc-modified-id="Going-up-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Going up</a></span></li><li><span><a href="#Going-next" data-toc-modified-id="Going-next-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Going next</a></span></li><li><span><a href="#Going-previous" data-toc-modified-id="Going-previous-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Going previous</a></span></li><li><span><a href="#Going-down" data-toc-modified-id="Going-down-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Going down</a></span></li><li><span><a href="#The-first-line" data-toc-modified-id="The-first-line-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>The first line</a></span></li><li><span><a href="#Prime" data-toc-modified-id="Prime-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Prime</a></span></li><li><span><a href="#Variant" data-toc-modified-id="Variant-17"><span class="toc-item-num">17&nbsp;&nbsp;</span>Variant</a></span></li><li><span><a href="#Modifier" data-toc-modified-id="Modifier-18"><span class="toc-item-num">18&nbsp;&nbsp;</span>Modifier</a></span></li><li><span><a href="#Full-grapheme-overview" data-toc-modified-id="Full-grapheme-overview-19"><span class="toc-item-num">19&nbsp;&nbsp;</span>Full grapheme overview</a></span></li><li><span><a href="#Search" data-toc-modified-id="Search-20"><span class="toc-item-num">20&nbsp;&nbsp;</span>Search</a></span></li><li><span><a href="#Explore-additional-data" data-toc-modified-id="Explore-additional-data-21"><span class="toc-item-num">21&nbsp;&nbsp;</span>Explore additional data</a></span></li><li><span><a href="#Add-your-own-data" data-toc-modified-id="Add-your-own-data-22"><span class="toc-item-num">22&nbsp;&nbsp;</span>Add your own data</a></span></li><li><span><a href="#Export-to-Emdros-MQL" data-toc-modified-id="Export-to-Emdros-MQL-23"><span class="toc-item-num">23&nbsp;&nbsp;</span>Export to Emdros MQL</a></span></li></ul></div>

<img align="right" src="images/quad.png" width="300"/>
<img align="right" src="images/tf-small.png"/>


# Tutorial

This notebook gets you started with using
[Text-Fabric](https://github.com/Dans-labs/text-fabric) for coding in cuneiform tablet transcriptions.

## What is Text-Fabric?

### Cuneiform tablets in ATF

Cuneiform tablets have been transcribed in ATF files, in which the marks on a tablet are represented
by ascii characters. The marks on a tablet have structure (they can be composed, they build cases and lines) and properties (they can be uncertain).

When you search for tablet data in an ATF file, you can do so conveniently by using regular expressions.

However, the ATF transcriptions have become packed with information. Not every transcriber uses ATF in the same way, and there are a few coding errors in the sources.

That means that the most obvious search expressions will leave out cases. Either you live with that, or you refine your search expressions.

An other issue is, that when you look for something, your search expressions must reflect the shape of not
only your target, but also everything else. There is virtually no separation of concerns.

### Text-Fabric

Text-Fabric is a model for textual data with annotations that is optimized for efficient data analysis. Not only that, it also facilitates the creation of new, derived data, which can be added to the original data.
Data combination is a feature of Text-Fabric.

Text-Fabric is being used for the
[Hebrew Bible](https://github.com/ETCBC/bhsa)
and a large body of linguisitic annotations on top of it. The researchers of the 
[ETCBC](http://etcbc.nl) thought that a plain database is not a satisfactory text model,
and that XML is too limited too express multiple hierarchies in a text smoothly.

That's why they adopted a model by
[Doedens](http://books.google.nl/books?id=9ggOBRz1dO4C)
that reflects more of the essential properties of text (sequence, embedding). This model is the basis of MQL, a working text-database system.
Text-Fabric is based on the same
[model](https://github.com/Dans-labs/text-fabric/wiki/Data-model),
and once the data is in Text-Fabric, it can be exported to MQL.

See more on the effort of modeling the Hebrew Bible in Dirk's article
[The Hebrew Bible as Data: Laboratory - Sharing - Experiences](https://doi.org/10.5334/bbi.18)

With data in Text-Fabric, it becomes possible to build rich online interfaces on the data of ancient texts.
For the Hebrew Bible, we have built
[SHEBANQ](https://shebanq.ancient-data.org).

Working with TF is a bit like buying from IKEA. You get your product in bits and pieces, and you assemble it yourself. TF decomposes any dataset into its components, nicely stacked per component, with every component uniquely labeled. You go to the store, make your selection, enter the warehouse, collect your parts, and, at home, assemble your product.

In order to enjoy an IKEA product, you do not need to be a craftsman, but you do need to be able to handle a screw driver.

In the TF world, it is the same. You do not have to be a professional programmer, but you do need to be able to program little things. A first course in Python is enough.

Another parallel: in IKEA you take a package with components home, and there you assemble it. 
In TF it is likewise: you download the TF data, and then you write a little program. Inside that program you can call up the Text-Fabric tool, which act as the IKEA user manual. But your program takes control, not Text-Fabric.

The best environment to enjoy Text-Fabric is in Python programs that you develop in a 
[Jupyter Notebook](http://jupyter.readthedocs.io/en/latest/).
This tutorial is such a notebook. If you are reading it online, you see text bits and code bits,
but you cannot execute the code bits.

If you download this tutorial, and you have installed Python, Jupyter, and Text-Fabric,
you can *execute* the code bits.

## Overview

* we tell you how to get Text-Fabric on your system
* we tell you how to get a set of cuneiform tablet transcriptions on your system
* we show how to explore the data:
  * finding the relevant nodes
  * moving from one place to an other
  * collecting the relevant information
  * perform analysis
  * visualize your results

## More information
Chances are that a bit of reading about the underlying
[data model](https://github.com/Dans-labs/text-fabric/wiki/Data-model)
helps you to follow the exercises below, and vice versa.

We have checked the conversion from the transcriptions to Text-Fabric extensively.
Cruelly, you might say. You can follow the checks
in a separate notebook [checks](checks.ipynb).

## Installing Text-Fabric

### Prerequisites

You need to have Python on your system. Most systems have it out of the box,
but alas, that is python2 and we need at least python 3.6.

Install it from [python.org]() or from [Anaconda]().
If you got it from python.org, you also have to install [Jupyter]().

### TF itself

```
pip install text-fabric
```

if you have installed Python with the help of Anaconda, or

```
sudo -H pip3 install text-fabric
```
if you have installed Python from [python.org](https://www.python.org).

###### Execute: If all this is done, the following cells can be executed.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys, os, collections
from tf.fabric import Fabric
from utils import Compare

### Cuneiform data

We have prepared a corpus of 6000 tablets, from the Uruk-III/IV period in Text-Fabric.
We have downloaded the transcriptions from CDLI, and converted them to Text-Fabric.

You can get the results from GitHub as follows.

We suggest you make an appropriate directory in your home directory:

```
github/Dans-labs
```

then go to that directory in a terminal, and then say

```
git clone https://github.com/Dans-labs/Nino-cunei
```

After that your directory structure shold look like this:

    your home direcectory\
    |                     - github\
    |                       |      - Dans-labs\
    |                       |        |         - Nino-cunei
    
#### Tip
If you start computing with this tutorial, first copy its parent directory to somewhere else,
outside your `Nino-cunei` directory.
If you pull changes from the `Nino-cunei` repository later, your work will not be overwritten.
Where you put your tutorial directory is up till you.
It will work from any directory.

###### Execute: it this has been done, you can execute the next cells

In [5]:
REPO = '~/github/Dans-labs/Nino-cunei'
SOURCE = 'uruk'
VERSION = '0.1'
CORPUS = f'{REPO}/tf/{SOURCE}/{VERSION}'
SOURCE_DIR = os.path.expanduser(f'{REPO}/sources/cdli')
TEMP_DIR = os.path.expanduser(f'{REPO}/_temp')
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

This is Text-Fabric 3.2.0
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

33 features found and 0 ignored


## Load Data

The transcriptions of the tablets in their TF form is organized in a model of nodes, edges and features.

The things such as tablets, faces, columns, cases, and, at the most basic level, signs, are numbered.
The signs correspond to number 1 ... ca. 120,000, in the same order as they occur in the corpus.
All other things are built from signs. They have number from ca 120,000 to 450,000.

In TF, we call these numbers *nodes*. Like a barcode, this number gives access to a whole bunch of
information about the corresponding object.

For example, lines have a property (in TF we call it a *feature*) called `fullNumber`. 
It contains the hierarchical number found at the start of the line in the transcription.

If the node for a line is `n`, we can find its hierarchical number by saying

```
F.fullNumber.v(n)
```

In words, it reads as:

* `F`: I want to look up a `F`eature
* `fullNumber`: the name of the feature
* `.v`: I want the value of that feature
* `(n)`: for the given node `n`

Seen in this way, the data is like a gigantic spreadsheet of 450,000 rows (the nodes),
with 30 columns (the features).

There is a bit more to it, since the nodes can be grouped together in ways we will see later on.

###### Execute: the next cell loads most features

In [6]:
api = TF.load('''
    grapheme prime repeat
    variant variantOuter
    modifier modifierInner modifierFirst
    damage uncertain remarkable written
    kind
    period name type identifier catalogId
    number fullNumber origNumber badNumbering
    crossref text
    srcLn srcLnNum
    op sub comments''')
api.makeAvailableIn(globals())
COMP = Compare(api, SOURCE_DIR, TEMP_DIR)

  0.00s loading features ...
   |     0.00s B catalogId            from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.03s B fullNumber           from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.03s B number               from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.08s B grapheme             from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.06s B srcLn                from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.03s B srcLnNum             from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B prime                from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B repeat               from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.01s B variant              from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B variantOuter         from /Users/dirk/github/Dans-labs/Nino-cunei/tf/uruk/0.1
   |     0.00s B modi

## A tour around the TF API

We have the data of over 6000 tablets at our fingertips now. But how can we see it?
In a meaningful way?

The following tour shows you the most important parts of the Text-Fabric API.
It is a set of functions, defined in the TF package, that you can use in your own program.

This notebook itself is our current program, and we have just loaded the API.

See in the next cells, how we navigate to the tablets we want and pick our data from them.

### Sections: tablet - column - line

Let's start with a single, specific tablet P448702.

Here is its transcription, copied from the CDLI source.

```
&P448702 = www archaeo-auction 004 
#atf: lang qpc 
@tablet 
@obverse 
@column 1 
$ beginning broken 
1'. [n] , [...] KA~a 
2.a'. [n] 2(N14) 3(N01) , KASZ~b NUN~a 
2.b'. 3(N01) , KASZ~a? GI 
3'. [n] 6(N14)#? 4(N01) 2(N39~a) , X [...] 
@column 2 
$ beginning broken 
$ blank space 
1'. , U4 |U4x1(N01)| SUKUD@inversum? NA 
@column 3 
$ beginning broken 
$ blank space 
@reverse 
$ (not imaged) 
```

There is a node for this tablet. How do we find it?

The easiest way is to ask TF to find the node on the basis of a section specification.

When we converted the transcriptions to TF, we made it so that tablets are the first
section level, columns the second, and lines the third. 

In order to find the node corresponding to a section, we have to give a tuple
`(tabletName, columnNumber, lineNumber)` to the appropriate function in TF.

Here you may leave out `lineNumber` (to specify a column), or both `columnNumber` and `lineNumber` 
(to specify a tablet).
The latter is our case here.

In [7]:
exampleTablet = T.nodeFromSection(('P448702',))
exampleTablet

150865

Ok, this is the node, the barcode, as it were. It gives you access to all data we have stored of this
tablet.

One of the things we have stored for tablets, columns, and lines, is the original transcription line.
Let's retrieve it.

In [9]:
F.srcLn.v(exampleTablet)

'&P448702 = www archaeo-auction 004 '

This is the source line of the tablet header.
How do we get the other source lines?

### Navigation

We'll need another concept first.
Things on a tablet have spatial relationships. 
While the transcriptions abstract from the physical relationships, there are still a few
important ones left: sequence and embedding.

At the most basic level we have signs. 
For all other levels, TF knows exactly which signs are part of the objects of those levels.

And based on that knowledge, we can travel from objects to the objects embedded in it, and back,
and to the right and left siblings.

All these functions start with `L.`, and they take a starting node as argument.

* `L.d()` goes "down": from enbedder to embeddee;
* `L.u()` goes "up": from embeddee to embedder;
* `L.p()` goes "previous": to the first left sibling;
* `L.n()` goes "next": to the first left sibling.

We can now find all the nodes contained in the example tablet.
When we retrieve them, we also fetch their *type*: is it a face, a column, a sign?

In [11]:
for child in L.d(exampleTablet):
    print(f'{child:>6} of type {F.otype.v(child)}')

180739 of type comment
    56 of type sign
157262 of type face
166709 of type column
180740 of type comment
    57 of type sign
229518 of type line
266443 of type case
196542 of type cluster
    58 of type sign
196543 of type cluster
    59 of type sign
    60 of type sign
229519 of type line
266444 of type case
266445 of type case
196544 of type cluster
    61 of type sign
    62 of type sign
    63 of type sign
    64 of type sign
    65 of type sign
266446 of type case
266447 of type case
    66 of type sign
    67 of type sign
    68 of type sign
229520 of type line
266448 of type case
196545 of type cluster
    69 of type sign
    70 of type sign
    71 of type sign
    72 of type sign
    73 of type sign
196546 of type cluster
    74 of type sign
166710 of type column
180741 of type comment
    75 of type sign
180742 of type comment
    76 of type sign
229521 of type line
266449 of type case
    77 of type sign
146957 of type quad
    78 of type sign
    79 of type sign
    80 of

A few things to note:

* the numbers are not really important to us, only to the program.
  Soon, we will stop showing them altogether
* the order in which the nodes are delivered, is a natural one:
  first things first, and bigger things before the smaller things they contain.

#### Source lines

So in order to find all the source lines of this tablet, we can just take all these nodes,
see whether they have a value for feature *srcLn*, retrieve it, an print it.

Let's do it, and write this as a function, i.e. a recipe that we can apply to every tablet.

In [12]:
def getSource(tablet):
    sourceLines = []
    sourceLines.append(F.srcLn.v(tablet))
    for child in L.d(tablet):
        sourceLine = F.srcLn.v(child)
        if sourceLine:
            sourceLines.append(sourceLine)
    return sourceLines

This function gets all source lines in a list.
Let's apply it to our sample tablet, and print out the list:

In [13]:
sourceLines = getSource(exampleTablet)
print('\n'.join(sourceLines))

&P448702 = www archaeo-auction 004 
#atf: lang qpc 
@obverse 
@column 1 
$ beginning broken 
1'. [n] , [...] KA~a 
2.a'. [n] 2(N14) 3(N01) , KASZ~b NUN~a 
2.b'. 3(N01) , KASZ~a? GI 
3'. [n] 6(N14)#? 4(N01) 2(N39~a) , X [...] 
@column 2 
$ beginning broken 
$ blank space 
1'. , U4 |U4x1(N01)| SUKUD@inversum? NA 
@column 3 
$ beginning broken 
$ blank space 
@reverse 
$ (not imaged) 


This is nearly the original. The only difference we spot is that we have left out
the `@tablet` line. This has been done because the `&P` line is the true header of the
tablet. In fact, the `@tablet` line is not consistently present in the source.

As you seen, the TF data is a bit more streamlined than the source transcriptions.

#### Source lines: refined

If you just want the lines, and nothing else in between, you can instruct the `L.d` function to
let through only nodes of a certain type.

Let's improve the `getSource` function to give it an additional, optional argument that
tells us the kind of object we want to see.

In [14]:
def getSource(tablet, nodeType=None):
    sourceLines = []
    sourceLines.append(F.srcLn.v(tablet))
    for child in L.d(tablet, nodeType):
        sourceLine = F.srcLn.v(child)
        if sourceLine:
            sourceLines.append(sourceLine)
    return sourceLines

If we call it without node type, we get the same results.

Let's call it with `nodeType='face'`:

In [17]:
sourceLines = getSource(exampleTablet, nodeType='face')
print('\n'.join(sourceLines))

&P448702 = www archaeo-auction 004 
@obverse 
@reverse 


We see only the face specifiers.

Now let's grab the lines: 

In [19]:
sourceLines = getSource(exampleTablet, nodeType='case')
print('\n'.join(sourceLines))

&P448702 = www archaeo-auction 004 
1'. [n] , [...] KA~a 
2.a'. [n] 2(N14) 3(N01) , KASZ~b NUN~a 
2.b'. 3(N01) , KASZ~a? GI 
3'. [n] 6(N14)#? 4(N01) 2(N39~a) , X [...] 
1'. , U4 |U4x1(N01)| SUKUD@inversum? NA 


#### Lines and cases

Note that we actually supplied `nodeType='case'`.
This is because we have modelled the data in such a way, that a line corresponds to all
transcription lines in a column that start with the same number.
Lines are divided in cases and sub-cases according to the hierarchical line numbers.
At the lowest level of sub-case we encounter the material that belong to single
transcription lines.

Also when lines only have a single case, we will have a case embedded in the line.

#### Line numbers

The TF data also has the original line numbers. Let us once again improve the `getSource` function by 
giving it an optional argument for fetching line numbers too:

In [30]:
def getSource(tablet, nodeType=None, lineNumbers=False):
    sourceLines = []
    if lineNumbers:
        sourceLines.append(f'{F.srcLnNum.v(tablet):>5}: {F.srcLn.v(tablet)}')
    for child in L.d(tablet, nodeType):
        sourceLine = F.srcLn.v(child)
        lineNumber = ''
        if sourceLine:
            if lineNumbers:
                lineNumber = f'{F.srcLnNum.v(child):>5}: '
            sourceLines.append(f'{lineNumber}{sourceLine}')
    return sourceLines

Now we can print the lines with source line numbers, so that we can look them up in the
source easily:

In [31]:
sourceLines = getSource(exampleTablet, nodeType='case', lineNumbers=True)
print('\n'.join(sourceLines))

   50: &P448702 = www archaeo-auction 004 
   56: 1'. [n] , [...] KA~a 
   57: 2.a'. [n] 2(N14) 3(N01) , KASZ~b NUN~a 
   58: 2.b'. 3(N01) , KASZ~a? GI 
   59: 3'. [n] 6(N14)#? 4(N01) 2(N39~a) , X [...] 
   63: 1'. , U4 |U4x1(N01)| SUKUD@inversum? NA 


You'll appreciate the way you can build up your own power tools as you go.
The functions of TF provide a foundation, so that start when all the gory details
have been taken care of.

### Mass operations

We have a mass of data, and we want to do meaningful things with that mass.
Here are a few ways to address all nodes, and significant subsets of all nodes.

### Walking all nodes

The generator `N()` provides you with all nodes, one by one, in a sequence.
They come in the natural order, as we have seen above:

* first things first
* bigger things before the smaller things they contain.

Let's print the first 20 nodes, together with their types.

In [33]:
LIMIT = 20

times = 0
for n in N():
    print(f'{n:>6} : {F.otype.v(n)}')
    times += 1
    if times >= 20:
        break

150862 : tablet
180733 : comment
     1 : sign
180734 : comment
     2 : sign
157258 : face
166700 : column
229498 : line
266423 : case
196539 : cluster
     3 : sign
     4 : sign
     5 : sign
166701 : column
229499 : line
266424 : case
     6 : sign
     7 : sign
     8 : sign
196540 : cluster


### All signs

Suppose we are only interested in the signs, not how they are organized in cases, lines,
columns, faces and tablets.

Suppose we just want to count them.

We could use `N()` as follows:

In [34]:
times = 0
for n in N():
    if F.otype.v(n) != 'sign':
        continue
    times += 1
print(times)

146955


In fact, TF makes it a bit easier to get to all sign nodes, or nodes from whatever type.

We have already used the `F.feature.v(n)` idiom.
But you can do more with features, beyond fetching particular values.

You can also retrieve the sequence of nodes that support a particular value.

We want to retrieve all nodes that support the value `sign` in their `otype` feature.

Here we go, and note that we use `.s(value)` instead of `.v(node)`.

In [35]:
signs = F.otype.s('sign')
len(signs)

146955

Signs have a feature `grapheme`, filled with the name of the sign, without variants (allographs), modifiers, and flags. Let's see the first 20 graphemes.

In [37]:
for sign in signs[0:20]:
    print(f'{sign:>2} : {F.grapheme.v(sign)}')

 1 : 
 2 : 
 3 : …
 4 : X
 5 : X
 6 : N14
 7 : X
 8 : SANGA
 9 : …
10 : 
11 : 
12 : X
13 : X
14 : X
15 : X
16 : X
17 : X
18 : X
19 : X
20 : X


A few things to note:

* there are empty signs
* there is the sign `…`

During conversion, we have put empty signs in metadata lines and rulings.
That is the mechanism by which TF knows where in the corpus they are positioned.
The do not belong to the material that corresponds to tablet content, except maybe in the
case of rulings.

The sign `…` originates from `...` in transcriptions, which is mostly surrounded by
`[ ]`. The brackets signify uncertainty, and can be put around other signs as well,
even sequences of signs. These brackets denote *clusters*, a node type that we will encounter
later.

So, the expression `[...]` in the source translates to a *cluster* in TF, with just one
sign in it, which has *grapheme* value `…`. 
Of course, this is not a real grapheme, and it will be easy to weed out such graphemes if
you do not need to take them into account. 

### Particular signs

Suppose we want to get all occurrences of the grapheme `SANGA`.

We can use the same trick as for getting the `signs`: we ask for the nodes that support
the value `SANGA` in their `grapheme` feature.

In [38]:
sangas = F.grapheme.s('SANGA')
len(sangas)

747

As an example of what you can do with such a set, let's pick only those SANGAs that
do not have a variant. We want `SANGA`, but not `SANGA~a`.

In [42]:
pureSangas = [s for s in sangas if F.variant.v(s) is None]
len(pureSangas)

0

Ah, all sangas are an allograph in this corpus. Are they all `~a`?

In [43]:
aSangas = [s for s in sangas if F.variant.v(s) == 'a']
len(aSangas)

729

Not all of them, let's see what other variants occur:

In [49]:
sangaVariants = set()
otherSangas = []

for s in sangas:
    if s not in aSangas:
        otherSangas.append(s)
        variant = F.variant.v(s)
        sangaVariants.add(variant)
        print(f'{s:>6} SANGA~{variant}')
print(f'All variants: {", ".join(sorted(sangaVariants))}')

 65752 SANGA~b
122525 SANGA~b
122536 SANGA~b
122577 SANGA~b
122593 SANGA~b
123008 SANGA~b
123011 SANGA~b
123019 SANGA~b
123025 SANGA~b
123052 SANGA~b
124094 SANGA~b
124783 SANGA~b
127407 SANGA~b
133550 SANGA~b
133681 SANGA~c
135991 SANGA~b
138239 SANGA~b
142282 SANGA~c
All variants: b, c


### From instance to context

We want to know where those other SANGA variants are.

Earlier we saw how we could get nodes if we knew the section.
Here we need the opposite. We know a few nodes, and want to get the section.
TF has a function for that:

In [50]:
for s in otherSangas:
    print(T.sectionFromNode(s))

('P273433', 'obverse:2', '3')
('P000884', 'obverse:1', '2')
('P000884', 'obverse:2', '1')
('P000886', 'obverse:2', '2')
('P000888', 'obverse:1', '1')
('P000958', 'obverse:2', '3')
('P000958', 'obverse:2', '4')
('P000958', 'obverse:2', '7')
('P000958', 'obverse:3', '2')
('P000958', 'obverse:5', '2')
('P001078', 'obverse:2', '2')
('P001161', 'obverse:1', '1')
('P001377', 'obverse:3', '3')
('P002079', 'obverse:1', '1')
('P002207', 'obverse:1', '1')
('P003233', 'obverse:2', '2')
('P000121', 'obverse:2', '3')
('P003454', 'obverse:2', '2')


If we want to see the source code line of these sangas, we can do it by what we have learned so far:

In [51]:
for s in otherSangas:
    for case in L.u(s, otype='case'):
        srcLine = F.srcLn.v(case)
        if srcLine is not None:
            print(f'{F.srcLnNum.v(case):>5}: {srcLine}')

53330: 3.b. 2(N14) 6(N01) , SANGA~b? 
 1292: 2. 2(N01) , SANGA~b MA
 1296: 1. 1(N01)# , X SANGA~b#?
 1322: 2. 1(N01)# , ZATU713# SANGA~b#
 1339: 1. [...] 2(N01)# , HASZHUR# SANGA~b# [...]
 1786: 3. 1(N01) , UMUN2# SANGA~b#
 1787: 4. 1(N01) , ZATU678# SANGA~b?
 1790: 7. 1(N01) , SUG5 SANGA~b
 1793: 2. 1(N01) , BU~a SANGA~b
 1807: 2. 1(N01)# , SANGA~b#
 2687: 2. 3(N01)# [...] , SANGA~b# KUR@g~a# [...]
 3280: 1. [...] 3(N01)# , SANGA~b
 5564: 3. [...] 1(N01) , SANGA~b#
10490: 1. 2(N14)#? , SANGA~b
10622: 1. , SANGA~c GIR3~b DUB~c
12763: 2. 1(N52)# , SANGA~b
14583: 3'. , SANGA~b? IB~b 
18084: 2. 1(N34) , SANGA~c 1(N58)


## End

Here ends the reburbished tutorial.
Below are fragments of the old tutorial.
These will be gone or reworked in the final version.

The grapheme name of each sign is a column `grapheme` in that spreadsheet.

The information whether a sign is damaged, constitutes a column `damaged`.

The corpus contains over 20 columns, not only for the signs, but also for a 150,000+ more
textual objects, such as *(sub)quads*, *clusters*, *lines*, *cases*, *columns*, *faces* and *tablets*.

We also have features that contain the original lines of transcription.
These features are filled for tablets, faces, columns, lines, and comments.

Instead of putting that information in one big table, the data is organized in separate columns.
We call those columns **features**.

We just load the features we need for this tutorial.
Later on, where we use them, it will become clear what they mean.

The result of this all is that we have a bunch of special variables at our disposal
that give us access to the text and data of the tablets.

At this point it is helpful to throw a quick glance at the text-fabric
[API documentation](https://github.com/Dans-labs/text-fabric/wiki/Api)
especially the right side bar.

The most essential thing for now is that we can use `F` to access the data in the features
we've loaded.
But there is more, such as `N`, which helps us to walk over the text, as we see in a minute.

# Counting

In order to get acquainted with the data, we start with the simple task of counting.

## Count all nodes
We use the 
[`N()` generator](https://github.com/Dans-labs/text-fabric/wiki/Api#walking-through-nodes)
to walk through the nodes.

We compared the tablet data to a gigantic spreadsheet, where the rows correspond to the words.
In Text-Fabric, we call the rows `slots`, because they are the textual positions that can be filled with words.

We also mentioned that there are also other textual objects. 
They are the tablets, columns, lines, etc.
They also correspond to rows in the big spreadsheet.

In Text-Fabric we call all these rows *nodes*, and the `N()` generator
carries us through those nodes in the textual order.

Just one extra thing: the `info` statements generate timed messages.
If you use them instead of `print` you'll get a sense of the amount of time that 
the various processing steps typically need.

In [5]:
indent(reset=True)
info('Counting nodes ...')

i = 0
for n in N(): i += 1

info('{} nodes'.format(i))

  0.00s Counting nodes ...
  0.08s 318107 nodes


Here you see it: more than 400,000 nodes!

## What are all those nodes?
Every node has a type, like sign, or line, face.
We know that we have many of them,
but what exactly are they?

Text-Fabric has two special features, `otype` and `oslots`, that must occur in every Text-Fabric data set.
`otype` tells you for each node its type, and you can ask for the number of `slot`s in the text.

Here we go!

In [6]:
F.otype.slotType

'sign'

In [7]:
F.otype.maxSlot

146955

In [8]:
F.otype.maxNode

318107

In [9]:
F.otype.all

('tablet',
 'face',
 'column',
 'line',
 'case',
 'cluster',
 'quad',
 'comment',
 'sign')

We can obtain a bit more knowledge about the types of nodes.

In [10]:
for (nodeType, avLen, startNode, endNode) in C.levels.data:
    print(f'{nodeType:<8} average length {avLen:>7.4f} from {startNode:>6} to {endNode:>6}')

tablet   average length 22.9761 from 150862 to 157257
face     average length 14.3466 from 157258 to 166699
column   average length  9.4888 from 166700 to 180732
line     average length  3.5643 from 229498 to 266422
case     average length  3.1821 from 266423 to 318107
cluster  average length  1.0313 from 196539 to 229497
quad     average length  2.0507 from 146956 to 150861
comment  average length  1.0000 from 180733 to 196538
sign     average length  1.0000 from      1 to 146955


Here is the first *cluster*.

In [11]:
cl = F.otype.s('cluster')[0]
cl

196539

This is what is embedded in it.

In [12]:
for n in L.d(cl):
    print(f'node {n:>6} of type {F.otype.v(n)}')

node      3 of type sign


Here is the third *sign*.

In [13]:
F.otype.v(3)

'sign'

And this is its context.

In [14]:
for n in L.u(3):
    print(f'node {n:>6} of type {F.otype.v(n)}')

node 196539 of type cluster
node 266423 of type case
node 229498 of type line
node 166700 of type column
node 157258 of type face
node 150862 of type tablet


## Count individual object types
This is an intuitive way to count the number of nodes in each type.
Note in passing, how we use the `indent` in conjunction with `info` to produce neat timed 
and indented progress messages.

In [15]:
indent(reset=True)
info('counting objects ...')

for otype in F.otype.all:
    i = 0

    indent(level=1, reset=True)

    for n in F.otype.s(otype): i+=1

    info('{:>7} {}s'.format(i, otype))

indent(level=0)
info('Done')

  0.00s counting objects ...
   |     0.00s    6396 tablets
   |     0.00s    9442 faces
   |     0.01s   14033 columns
   |     0.00s   36925 lines
   |     0.01s   51685 cases
   |     0.01s   32959 clusters
   |     0.00s    3906 quads
   |     0.00s   15806 comments
   |     0.02s  146955 signs
  0.09s Done


# Locality

We travel upwards and downwards, forwards and backwards through the nodes.
The Locality-API (`L`) provides functions: `u()` for going up, and `d()` for going down,
`n()` for going to next nodes and `p()` for going to previous nodes.

These directions are indirect notions: nodes are just numbers, but by means of the
`oslots` feature they are linked to slots. One node *contains* an other node, if the one is linked to a set of slots that contains the set of slots that the other is linked to.
And one if next or previous to an other, if its slots follow of precede the slots of the other one.

`L.u(node)` **Up** is going to nodes that embed `node`.

`L.d(node)` **Down** is the opposite direction, to those that are contained in `node`.

`L.n(node)` **Next** are the next *adjacent* nodes, i.e. nodes whose first slot comes immediately after the last slot of `node`.

`L.p(node)` **Previous** are the previous *adjacent* nodes, i.e. nodes whose last slot comes immediately before the first slot of `node`.

All these functions yield nodes of all possible otypes.
By passing an optional parameter, you can restrict the results to nodes of that type.

The result are ordered according to the order of things in the text.

The functions return always a tuple, even if there is just one node in the result.

## Going up
We go from the first sign to the tablet it contains.
Note the `[0]` at the end. You expect one tablet, yet `L` returns a tuple. 
To get the only element of that tuple, you need to do that `[0]`.

If you are like me, you keep forgetting it, and that will lead to weird error messages later on.

In [16]:
firstTablet = L.u(1, otype='tablet')[0]
print(firstTablet)

150862


And let's see all the containing objects of sign 100:

In [17]:
w = 100
for otype in F.otype.all:
    if otype == F.otype.slotType: continue
    up = L.u(w, otype=otype)
    upNode = None if len(up) == 0 else up[0]
    if upNode is None:
        print('sign {} is not contained in a {}'.format(w, otype))
    else:
        print('sign {} is contained in {} {}'.format(w, otype, upNode))

sign 100 is contained in tablet 150866
sign 100 is contained in face 157264
sign 100 is contained in column 166712
sign 100 is contained in line 229525
sign 100 is contained in case 266453
sign 100 is not contained in a cluster
sign 100 is not contained in a quad
sign 100 is not contained in a comment


## Going next
Let's go to the next nodes of the first tablet.

In [18]:
afterFirstTablet = L.n(firstTablet)
for n in afterFirstTablet:
    print('{:>7}: {:<13} first slot={:<6}, last slot={:<6}'.format(
        n, F.otype.v(n),
        E.oslots.s(n)[0],
        E.oslots.s(n)[-1],
    ))
secondTablet = L.n(firstTablet, otype='tablet')[0]

     10: sign          first slot=10    , last slot=10    
 180735: comment       first slot=10    , last slot=10    
 150863: tablet        first slot=10    , last slot=39    


## Going previous

And let's see what is right before the second tablet.

In [19]:
for n in L.p(secondTablet):
    print('{:>7}: {:<13} first slot={:<6}, last slot={:<6}'.format(
        n, F.otype.v(n),
        E.oslots.s(n)[0],
        E.oslots.s(n)[-1],
    ))

 150862: tablet        first slot=1     , last slot=9     
 157258: face          first slot=3     , last slot=9     
 166701: column        first slot=6     , last slot=9     
 229499: line          first slot=6     , last slot=9     
 266424: case          first slot=6     , last slot=9     
 196540: cluster       first slot=9     , last slot=9     
      9: sign          first slot=9     , last slot=9     


## Going down

We go to the columns of the second tablet, and just count them.

In [20]:
columns = L.d(secondTablet, otype='column')
print(len(columns))

5


## The first line
We pick the first line and the first sign, and explore what is above and below them.

In [21]:
firstLine = L.d(firstTablet, otype='line')[0]

for n in [1, firstLine]:
    indent(level=0)
    info('Node {}'.format(n), tm=False)
    indent(level=1)
    info('UP', tm=False)
    indent(level=2)
    info('\n'.join(['{:<15} {}'.format(u, F.otype.v(u)) for u in L.u(n)]), tm=False)
    indent(level=1)
    info('DOWN', tm=False)
    indent(level=2)
    info('\n'.join(['{:<15} {}'.format(u, F.otype.v(u)) for u in L.d(n)]), tm=False)
indent(level=0)
info('Done', tm=False)

Node 1
   |   UP
   |      |   180733          comment
   |      |   150862          tablet
   |   DOWN
   |      |   
Node 229498
   |   UP
   |      |   266423          case
   |      |   166700          column
   |      |   157258          face
   |      |   150862          tablet
   |   DOWN
   |      |   166700          column
   |      |   266423          case
   |      |   196539          cluster
   |      |   3               sign
   |      |   4               sign
   |      |   5               sign
Done


# Text

The `T` functions provide ways of printing out text, and they know about section levels.

We use section levels `tablet`, `column`, `line`.
`face` is a level of nodes, but not a section level.

We will define our own function to get the literal transcription text back for
tablets, faces, etc.

In [22]:
oLevels = '''
    tablet
    face
    column
    case
'''.strip().split()

lowerLevel = dict((oLevels[n], oLevels[n+1]) for n in range(len(oLevels) - 1))

def transObject(n):
    kind = F.otype.v(n)
    trans = []
    trans.append(f'{F.srcLnNum.v(n):>7}: {F.srcLn.v(n)}')
    for c in E.comments.f(n):
        trans.append(f'{F.srcLnNum.v(c):>7}: {F.srcLn.v(c)}')
    print('\n'.join(trans))
    subKind = lowerLevel.get(kind, None)
    if subKind:
        for m in L.d(n, otype=subKind):
            if F.srcLn.v(m) is not None:
                transObject(m)

In [23]:
transObject(firstTablet)

      1: &P006427 = HJN 0044
      2: #version: 0.1
      3: #atf: lang qpc
      4: @obverse
      5: @column 1
      6: 1. [...] , X X
      7: @column 2
      8: 1. 3(N14) X SANGA~a? [...]


If we know the *P-number*, we can get the tablet with that P-number by means of
`T.nodeFromSection()`.

You pass this function a tuple, representing *tablet*, *column*, *line*, and it gives you back
the node of the corresponding object.

*column* and *line* are optional.

In [24]:
tabletId = 'P471695'
tabletNode = T.nodeFromSection((tabletId,))
tabletNode

150867

Now we can get the transcription of this tablet.

In [25]:
transObject(tabletNode)

     87: &P471695 = Anonymous 0712 
     88: #atf: lang qpc 
     90: @obverse 
     91: @column 1
     92: 1.a. 3(N01) , APIN~a 3(N57) UR4~a 
     93: 1.b1. , (EN~a DU ZATU759)a 
     94: 1.b2. , (BAN~b KASZ~c)a 
     95: 1.b3. , (KI@n SAG)a 
     96: 2.a. 1(N14) 2(N01) , [...] 
     97: 2.b1. , (3(N57) PAP~a)a 
     98: 2.b2. , (SZU KI X)a 
     99: $ n lines broken  
    100: 2.b3'. , (EN~a AN EZINU~d)a 
    101: 2.b4'. , (IDIGNA [...])a 
    102: $ rest broken 
    103: $ (for a total of 12 sub-cases with PNN) 
    104: @column 2
    105: 1.a. 1(N01) , ISZ~a#? 
    106: 1.b1. , (PAP~a GIR3~c)a
    107: $ blank space 
    108: $ rest broken 
    109: @reverse 
    110: $ beginning broken 
    111: 1'. [1(N14)] 6(N01)# , [...] 
    111: 1'. [1(N14)] 6(N01)# , [...] 


# Graphemes

`F`
gives access to all features.
Every feature has a method
`freqList()`
to generate a frequency list of its values, higher frequencies first.
Here are the graphemes (the top 20):

In [26]:
for (value, frequency) in F.grapheme.freqList()[0:20]:
    print(f'{frequency:>5} x {value}')

29618 x …
21676 x N01
17128 x 
 6956 x X
 5924 x N14
 1970 x EN
 1846 x N57
 1835 x N34
 1349 x SZE
 1241 x GAL
 1125 x DUG
 1069 x AN
 1046 x U4
  892 x NUN
  881 x SAL
  879 x PAP
  877 x E2
  875 x GI
  788 x BA
  747 x SANGA


We can do a bit more: we'll write a file with all graphemes to your TEMP_DIR.
In fact, we'll write two: one ordered by grapheme, and one ordered by frequency.

In order to not clutter this notebook, we use a function `writeFreqs()`, defined in 
[utils](utils.py) in the same directory.

In [27]:
COMP.writeFreqs('grapheme-plain', F.grapheme.freqList(), 'bare grapheme')

There are 667 bare graphemes


Now have a look at your TEMP_DIR and you see two generated files:

* `graphemes-plain-alpha.txt` (sorted by grapheme)
* `graphemes-plain-freq.txt` (sorted by frequency)

But we can do better, we also want the prime, variants, and modifiers taken into account.

Let us first see what they can be.

## Prime

The prime is a feature with two values: 1 or 0. 1 means: there is a prime.
Below you see how often that occurs.
Note that we count all primes here: on signs, case numbers and column numbers.

For more info and a check on the occurrences of primes, see [checks](checks.ipynb).

In [28]:
for (value, frequency) in F.prime.freqList():
    print(f'{frequency:>5} x {value}')

 5184 x 1


## Variant

The variant or allograph is what occurs after the grapheme and after the `~` symbol, which should be digits and/or
lowercase letters except the `x`.

Here is the frequency list of variant values.

In [29]:
for (value, frequency) in F.variant.freqList():
    print(f'{frequency:>5} x {value}')

23804 x a
 4172 x b
 1532 x c
 1356 x a1
  703 x b1
  194 x a2
  187 x d
  127 x b2
   85 x f
   73 x a3
   40 x e
   29 x c2
   22 x c1
   22 x c3
   17 x v
   14 x c5
   13 x b3
   12 x a0
   12 x d1
   11 x c4
    6 x a4
    6 x g
    5 x d2
    4 x d4
    4 x h
    2 x 3a
    2 x d3
    1 x h2


## Modifier

The modifier is what occurs after the grapheme and after the `@` symbol, which should be digits and/or
lowercase letters except the `x`.

Here is the frequency list of *modifier* and *rmodifier* values.

In [30]:
for (value, frequency) in F.modifier.freqList():
    print(f'{frequency:>5} x {value}')

  648 x g
  251 x t
   39 x n
    6 x r
    4 x s
    1 x c
    1 x v


In [32]:
for (value, frequency) in F.modifierInner.freqList():
    print(f'{frequency:>5} x {value}')

   25 x f
   15 x t
    1 x r
    1 x v


## Full grapheme overview

We make a frequency list of all full graphemes, i.e. the grapheme including variant, modifier, and prime.
We show as they appear in transcriptions.

First we show on what node types primes, variants and modifiers occur.
We only deal with cases where they occur on signs, ignoring the cases where they occur on (sub)quads.

In [33]:
for feature in ('prime', 'variant', 'modifier'):
    nodeTypes = collections.Counter()
    for n in N():
        if Fs(feature).v(n):
            nodeTypes[F.otype.v(n)] += 1
    for (value, frequency) in nodeTypes.items():
        print(f'{feature:<10}: {frequency:>5} x {value}')

prime     :  4652 x case
prime     :   523 x column
prime     :     9 x sign
variant   : 32455 x sign
modifier  :   950 x sign


Now the full graphemes.

In [34]:
fullGraphemes = collections.Counter()

for n in F.otype.s('sign'):
    fullGrapheme = COMP.strFromSign(n)
    fullGraphemes[fullGrapheme] += 1
    
for (value, frequency) in sorted(fullGraphemes.items(), key=lambda x: (-x[1], x[0]))[0:20]:
    print(f'{frequency:>5} x {value}')
    
COMP.writeFreqs('grapheme-full', fullGraphemes.items(), 'full grapheme')

29618 x ...
17128 x 
12996 x 1(N01)
 6956 x X
 3081 x 2(N01)
 2606 x 1(N14)
 1849 x EN~a
 1603 x 3(N01)
 1357 x 2(N14)
 1308 x SZE~a
 1304 x 5(N01)
 1224 x GAL~a
 1119 x 4(N01)
 1069 x AN
 1045 x U4
 1001 x 1(N34)
  881 x SAL
  874 x GI
  854 x PAP~a
  801 x 1(N57)
There are 1529 full graphemes


# Edge features: left and right

We have not talked about edges much. If the nodes correspond to the rows in the big spreadsheet,
the edges point from one row to another.

One edge we have encountered: the special feature `oslots`.
Each non-slot node is linked by `oslots` to all of its slot nodes.

An edge is really a feature as well.
Whereas a node feature is a column of information,
one cell per node, 
an edge feature is also a column of information, one cell per pair of nodes.

In the tablets quads may be subdivided into subquads and signs, related by operators.
If there is an operator *op* between `qLeft` and `qRight`, there is an 
edge between `qLeft` and `qRight` with feature `op` having value *op*.

And if a quad is the result of an operator working on operands, which are sub-*quads* or *signs*,
there will be edges between the big quad and its operands with feature `sub`, having no value.

Likewise, there will be edges between *lines* and *cases* and their subcases, also
having feature `sub` with no value.

# Next steps

By now you have an impression how to compute around in the Hebrew Bible.
While this is still the beginning, I hope you already sense the power of unlimited programmatic access
to all the bits and bytes in the data set.

Here are a few directions for unleashing that power.

## Search
Text-Fabric contains a flexible search engine, that does not only work for the BHSA data,
but also for data that you add to it.
There is a tutorial dedicated to [search](search.ipynb).
And if you already know MQL queries, you can build from that in
[searchFromMQL](searchFromMQL.ipynb).

## Explore additional data
The ETCBC has a few other repositories with data that work in conjunction with the BHSA data.
One of them you have already seen: 
[phono](https://github.com/ETCBC/phono),
for phonetic transcriptions.

There is also
[parallels](https://github.com/ETCBC/parallels)
for detecting parallel passages,
and
[valence](https://github.com/ETCBC/valence)
for studying patterns around verbs that determine their meanings.

## Add your own data
If you study the additional data, you can observe how that data is created and also
how it is turned into a text-fabric data module.
The last step is incredibly easy. You can write out every Python dictionary where the keys are numbers
and the values string or numbers as a Text-Fabric feature.
When you are creating data, you have already constructed those dictionaries, so writing
them out is just one method call.
See for example how the
[flowchart](https://github.com/ETCBC/valence/blob/master/programs/flowchart.ipynb#Add-sense-feature-to-valence-module)
notebook in valence writes out verb sense data.
![flow](images/valence.png)

You can then easily share your new features on GitHub, so that your colleagues everywhere 
can try it out for themselves.

## Export to Emdros MQL

[EMDROS](http://emdros.org), written by Ulrik Petersen,
is a text database system with the powerful *topographic* query language MQL.
The ideas are based on a model devised by Christ-Jan Doedens in
[Text Databases: One Database Model and Several Retrieval Languages](https://books.google.nl/books?id=9ggOBRz1dO4C).

Text-Fabric's model of slots, nodes and edges is a fairly straightforward translation of the models of Christ-Jan Doedens and Ulrik Petersen.

[SHEBANQ](https://shebanq.ancient-data.org) uses EMDROS to offer users to execute and save MQL queries against the Hebrew Text Database of the ETCBC.

So it is kind of logical and convenient to be able to work with a Text-Fabric resource through MQL.

If you have obtained an MQL dataset somehow, you can turn it into a text-fabric data set by `importMQL()`,
which we will not show here.

And if you want to export a Text-Fabric data set to MQL, that is also possible.

After the `Fabric(modules=...)` call, you can call `exportMQL()` in order to save all features of the
indicated modules into a big MQL dump, which can be imported by an EMDROS database.

# Clean caches

Text-Fabric pre-computes data for you, so that it can be loaded faster.
If the original data is updated, Text-Fabric detects it, and will recompute that data.

But there are cases, when the algorithms of Text-Fabric have changed, without any changes in the data, that you might
want to clear the cache of precomputed results.

There are two ways to do that:

* Locate the `.tf` directory of your dataset, and remove all `.tfx` files in it.
  This might be a bit awkward to do, because the `.tf` directory is hidden on Unix-like systems.
* Call `TF.clearCache()`, which does exactly the same.

It is not handy to execute the following cell all the time, that's why I have commented it out.
So if you really want to clear the cache, remove the comment sign below.

In [34]:
# TF.clearCache()